In [6]:
from tsfresh import extract_features
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm


frames = []
counter = 1
for file in tqdm(os.listdir('F:/lightcurves/')):
    filepath = 'F:/lightcurves/' + file
    with open(filepath,'rb') as rfile:
        lc = pickle.load(rfile)
    frames.append(pd.DataFrame({'id':counter*np.ones(len(lc.flux)),'time':lc.time,'flux':lc.flux}))
    counter = counter + 1
timeseries = pd.concat(frames)

100%|████████████████████████████████████████████████████████████████████████████| 15767/15767 [02:48<00:00, 93.74it/s]


In [ ]:
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction:   0%|                                                                       | 0/10 [00:00<?, ?it/s]

In [16]:
print(len(extracted_features.columns))
extracted_features['flux__mean']

763


id
1.0      8440.162238
2.0      5331.049356
3.0      6833.173367
4.0      8490.189776
5.0      5378.675228
            ...     
136.0    4905.288059
137.0    4856.394545
138.0    -760.834600
139.0    4670.863339
140.0    4759.945606
Name: flux__mean, Length: 140, dtype: float64

In [28]:
import matplotlib.pyplot as plt
n = 'mean'
b = 25
print(extracted_features.columns[n][6:])
data = extracted_features[extracted_features.columns[n]]
print(np.max(data)/np.std(data))
plt.hist(data,bins=b)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
extracted_features.to_csv('F:/catalogues/features.csv')

NameError: name 'extracted_features' is not defined

In [2]:
pip install tsfresh

Note: you may need to restart the kernel to use updated packages.
